<a href="https://colab.research.google.com/github/brandonko/FairnessNLP/blob/main/Bias_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bias Evaluation Metrics**

In [1]:
import math

### Co-occurrence Bias in the Dataset
Metric defined in [Qian et al. (2019)](https://arxiv.org/pdf/1905.12801.pdf), gender word lists from [Zhao et al. (2018)](https://arxiv.org/abs/1809.01496)

In [5]:
# Read in the gender word lists. Edit the file paths below to go to the
# files containing the female and male word lists. These word lists are
# in data/female_word_file.txt and data/male_word_file.txt in the GitHub repo.
female_words = []
male_words = []
with open('/content/drive/MyDrive/NLP Capstone/data/female_word_file.txt', 'r') as female_word_file:
    female_words = female_word_file.read().split()
with open('/content/drive/MyDrive/NLP Capstone/data/male_word_file.txt', 'r') as male_word_file:
    male_words = male_word_file.read().split()

In [11]:
def measure_cooccurrence_bias(data, female_words, male_words, window=10):
    """Measures the co-occurrence bias and conditional co-occurrence bias,
    as defined by Qian et al. (2019), of the given data, using the given
    lists of female and male words.

    Args:
        data: The dataset to measure bias in. Expected format is a list
        where each element is text.
        female_words: List of female gendered words.
        male_words: List of male_gendered words.
        window: An integer representing the max distance between a gendered
        word and a gender neutral word in the text in order to count those
        two words as co-occurring.

    Returns:
        The co-occurrence bias and conditional bias of the given data.
    """
    word_occur_counts = dict()
    num_male_words = 0
    num_female_words = 0
    for item in data:
        cur_tokens = item.lower().split(' ')
        for i in range(0, len(cur_tokens)):
            if cur_tokens[i] in female_words:
                num_female_words += 1
                start_index = max(0, i - window)
                stop_index = min(i + window, len(cur_tokens))
                for j in range(start_index, i):
                    if (cur_tokens[j] not in female_words) and (cur_tokens[j] not in male_words):
                        if cur_tokens[j] in word_occur_counts:
                            cur_count = word_occur_counts[cur_tokens[j]]
                            word_occur_counts[cur_tokens[j]] = (cur_count[0] + 1, cur_count[1])
                        else:
                            word_occur_counts[cur_tokens[j]] = (1, 0)
                for j in range(i + 1, stop_index):
                    if (cur_tokens[j] not in female_words) and (cur_tokens[j] not in male_words):
                        if cur_tokens[j] in word_occur_counts:
                            cur_count = word_occur_counts[cur_tokens[j]]
                            word_occur_counts[cur_tokens[j]] = (cur_count[0] + 1, cur_count[1])
                        else:
                            word_occur_counts[cur_tokens[j]] = (1, 0)
            elif cur_tokens[i] in male_words:
                num_male_words += 1
                start_index = max(0, i - window)
                stop_index = min(i + window, len(cur_tokens))
                for j in range(start_index, i):
                    if (cur_tokens[j] not in female_words) and (cur_tokens[j] not in male_words):
                        if cur_tokens[j] in word_occur_counts:
                            cur_count = word_occur_counts[cur_tokens[j]]
                            word_occur_counts[cur_tokens[j]] = (cur_count[0], cur_count[1] + 1)
                        else:
                            word_occur_counts[cur_tokens[j]] = (0, 1)
                for j in range(i + 1, stop_index):
                    if (cur_tokens[j] not in female_words) and (cur_tokens[j] not in male_words):
                        if cur_tokens[j] in word_occur_counts:
                            cur_count = word_occur_counts[cur_tokens[j]]
                            word_occur_counts[cur_tokens[j]] = (cur_count[0], cur_count[1] + 1)
                        else:
                            word_occur_counts[cur_tokens[j]] = (0, 1)
    cooccurrence_bias = 0
    conditional_cooccurrence = 0
    num_words = 0
    for word in word_occur_counts.keys():
        counts = word_occur_counts[word]
        if counts[0] + counts[1] > 20:
            if counts[0] != 0 and counts[1] != 0:
                num_words += 1
                cooccurrence_bias += abs(math.log(counts[1] / counts[0]))
                if num_male_words != 0 and num_female_words != 0:
                    prob_word_given_male = counts[1] / num_male_words
                    prob_word_given_female = counts[0] / num_female_words
                    conditional_cooccurrence += abs(math.log(prob_word_given_male / prob_word_given_female))
    if num_words > 0:
        cooccurrence_bias /= num_words
        conditional_cooccurrence /= num_words
    return (cooccurrence_bias, conditional_cooccurrence)

### Embedding Bias
Metric defined in [Qian et al. (2019)](https://arxiv.org/pdf/1905.12801.pdf)

### Counterfactual Evaluation of Sentiment Bias
Metric defined in [Huang et al. (2020)](https://aclweb.org/anthology/2020.findings-emnlp.7.pdf)

### (epsilon-k)-Fairness
Metric defined in [Ma et al. (2020)](https://www.semanticscholar.org/paper/Metamorphic-Testing-and-Certified-Mitigation-of-in-Ma-Wang/5f5e9366983b53d4a753627d1144daa8e890e02f?p2df)